In [16]:
import talib 
from talib import MA_Type
import pandas as pd
import datetime
import pandas as pd
import os
from urllib.request import urlopen
import json
import os 
import numpy as np
import tushare as ts
import datetime
from datetime import timedelta
import pyecharts.options as opts
from pyecharts.charts import Candlestick,Line
import matplotlib.pyplot as plt
import seaborn as sns
from alpha101 import *
pd.set_option('display.max_columns', 100)
import warnings
import tushare as ta
import joblib
warnings.filterwarnings("ignore")

In [17]:
# MACD顶底背离金叉与死叉
def MACD_Deviation(df,bottom=True,top=True):
    open_p = df['Open'].values
    high_p = df['High'].values
    close_p = df['Close'].values
    low_p = df['Low'].values
    volume = df["Volume"].values
    volume_p = df["Volume"].values
    df["MACD_DIF"],df["MACD_DEA"],df["MACD_Hist"] = talib.MACD(close_p, fastperiod=12, slowperiod=26, signalperiod=9)
    df['JCHA'] = np.where((df["MACD_Hist"] > 0) & (df["MACD_Hist"].shift(1) < 0),1,0) # type: np.ndarray
    df['SCHA']  = np.where((df["MACD_Hist"] < 0) & (df["MACD_Hist"].shift(1) > 0),1,0) # type: np.ndarray
    
    # 判断底背离
    jcha_data = df[df['JCHA']==1]
    jcha_data.reset_index(drop=True,inplace=True)

    jcha_data['pct_chg'] = 100*jcha_data['Close'].pct_change()
    jcha_data['DIF_chg'] = 100*jcha_data['MACD_DIF'].pct_change()
    jcha_data['FLAG']=None
    for i in range(1,len(jcha_data)):
        if i >=len(jcha_data):
            pass
        # 构造有效金叉
        elif (pd.to_datetime(jcha_data.loc[i,'Date'])-pd.to_datetime(jcha_data.loc[i-1,'Date'])).days < 20:
            jcha_data.drop(index=[i],inplace=True)
            jcha_data.reset_index(drop=True,inplace=True)
            if i >=len(jcha_data):
                pass
            else:
                date_0 = jcha_data.loc[i-1,'Date']
                date_1 = jcha_data.loc[i,'Date']
                df_1 = df[(df['Date']>=date_0) & (df['Date']<=date_1)]
                if len(df_1[df_1['MACD_DEA']>0]):
                    jcha_data.loc[i,'FLAG']=0
                else:
                    jcha_data.loc[i,'FLAG']=1
        else:
            date_0 = jcha_data.loc[i-1,'Date']
            date_1 = jcha_data.loc[i,'Date']
            df_1 = df[(df['Date']>=date_0) & (df['Date']<=date_1)]
            if len(df_1[df_1['MACD_DEA']>0]):
                jcha_data.loc[i,'FLAG']=0
            else:
                jcha_data.loc[i,'FLAG']=1
    jcha_data['JCHA_FLAG'] = np.where(( (jcha_data['pct_chg']<=2) & 
                                       ((abs(jcha_data['pct_chg']) >= abs(jcha_data['DIF_chg'])) | (jcha_data['MACD_DIF']>=jcha_data['MACD_DIF'].shift(1)))
                                       & (jcha_data['FLAG']==1)
                                      ),1,0)
    del jcha_data['FLAG']
    
    
    # 判断顶背离
    scha_data = df.loc[df['SCHA']==1]
    scha_data.reset_index(drop=True,inplace=True)
    scha_data['pct_chg'] = 100*scha_data['Close'].pct_change()
    scha_data['DIF_chg'] = 100*scha_data['MACD_DIF'].pct_change()
    scha_data['FLAG']=None
    # 有效的金叉与死叉的构建还存在问题，有效金叉是指MACDhist能有效稳定持续3天或者5天左右，如果小于3天，则认为是无效金叉/死叉
    for i in range(1,len(scha_data)):
        if i >=len(scha_data):
            pass
        # 构造有效死叉
        elif (pd.to_datetime(scha_data.loc[i,'Date'])-pd.to_datetime(scha_data.loc[i-1,'Date'])).days < 20:
            scha_data.drop(index=[i],inplace=True)
            scha_data.reset_index(drop=True,inplace=True)
            if i >=len(scha_data):
                pass
            else:
                date_0 = scha_data.loc[i-1,'Date']
                date_1 = scha_data.loc[i,'Date']
                df_1 = df[(df['Date']>=date_0) & (df['Date']<=date_1)]
                if len(df_1[df_1['MACD_DEA']<0]):
                    scha_data.loc[i,'FLAG']=0
                else:
                    scha_data.loc[i,'FLAG']=1
        else:
            date_0 = scha_data.loc[i-1,'Date']
            date_1 = scha_data.loc[i,'Date']
            df_1 = df[(df['Date']>=date_0) & (df['Date']<=date_1)]
            if len(df_1[df_1['MACD_DEA']<0]):
                scha_data.loc[i,'FLAG']=0
            else:
                scha_data.loc[i,'FLAG']=1

    scha_data['SCHA_FLAG'] = np.where(( (scha_data['pct_chg']>=-2) & 

                                    ((abs(scha_data['pct_chg']) >= abs(scha_data['DIF_chg'])) | (scha_data['MACD_DIF']<=scha_data['MACD_DIF'].shift(1)) )
                                       & (scha_data['FLAG']==1)
                                      ),1,0)
    del scha_data['FLAG']
    
    return jcha_data,scha_data

In [18]:
# 将tushare日期转换成固定格式
def str_to_date(int):
    date = str(int)
    if len(date) == 8:
        return date[0:4] + '-' + date[4:6] + '-' + date[6:8]
    else:
        return date

In [19]:
# 转回无 - 的str模式的date
def date_to_str(x):
    date = x
    if len(date) == 10:
        return date[0:4]  + date[5:7] + date[8:10]
    else:
        return date

In [20]:
# 获取正常上市的所有股票代码
def get_allstock_code(exchange='', list_status='L', fields='ts_code'):
    data = pro.stock_basic(exchange=exchange, list_status=list_status, fields=fields)
    return data

In [21]:
# 获取股票数据
def get_stock_data(code, start, end):
    stock_df = pro.daily(ts_code=code, start_date=start, end_date=end)
    stock_df['trade_date'] = stock_df['trade_date'].apply(str_to_date)
    stock_df.sort_values(['trade_date'],inplace=True)
    stock_df.reset_index(drop=True,inplace=True)
    return stock_df

In [22]:
# 股价复权+计算技术性指标因子
def fuquan(df):
    df['pct_chg'] = 100*(df['Close'] / df['pre_close'] - 1).round(4)
    # 股票复权
    df['fuquan'] = (1 + df['pct_chg']/100).cumprod()
    # close 是没复权的，Close是复权
    df['close'] = df['Close']
    df['Close'] = (df['fuquan'] * (df.iloc[0]['Close'] / df.iloc[0]['fuquan'])).round(2)
    
    df['Open'] = (df['Open'] / df['close'] * df['Close']).round(2)
    df['High'] = (df['High'] / df['close'] * df['Close']).round(2)
    df['Low'] = (df['Low'] / df['close'] * df['Close']).round(2)
    
    del df['close']
    
    # 分别计算当天之后 5 ，10 ，20天的涨跌幅
    df['pct_chg_5'] = (100*(df['Close'].shift(-5)/df['Close'] - 1)).round(2)
    df['pct_chg_10'] = (100*(df['Close'].shift(-10)/df['Close'] - 1)).round(2)
    df['pct_chg_20'] = (100*(df['Close'].shift(-20)/df['Close'] - 1)).round(2)
    df['pct_chg_30'] = (100*(df['Close'].shift(-30)/df['Close'] - 1)).round(2)
    
    # 计算90日bias
    df['mean_90'] = df['Close'].rolling(90).mean()
    df['bias_90'] = 100*(df['Close']/df['mean_90']-1)
    
    
    # 计算5日均线以及60日均线，当5日均线上穿60日均线，为买入信号
    df['mean_5'] = df['Close'].rolling(5).mean()
    df['mean_60'] = df['Close'].rolling(60).mean()
    df['5upcross60'] = 100*(df['mean_5']/df['mean_60']-1).round(4)
    
    # 计算MA60与MA90的偏离程度
    df['diff_MA60-MA90'] = 100*(df['mean_60']/df['mean_90'] - 1).round(4)
    
    del df['mean_5']
    del df['mean_60']
    del df['mean_90']
    # 计算成交量
    df['Vol_5'] = df['Volume'].rolling(5).mean()
    # 计算RSI
    close_p = df['Close'].values
    df["RSI"] = talib.RSI(close_p, timeperiod=14)
    # 计算当日最高点与收盘价的回撤
    df['now_high_close'] = 100*(df['Close']/df['High']-1).round(4)
    # 计算股票波动率
    df['std_60'] = round(100*(df['Close'].rolling(50).std() / df['Close'].rolling(50).mean()),4)
    
    # 30天内的最高点的回撤
    df['30_down'] = 100*(df['Close']/df['High'].shift(1).rolling(20).max()-1).round(4)
    
    # 计算一些alpha因子
    df,extra_agg_dict = get_pos_alpha(df, extra_agg_dict={})

    # 计算一年半内最高点与当日的一个最大回撤
#     df['max_down'] = 100*(df['Close']/df['Close'].rolling(365,min_periods=90).max()-1).round(4)
    
    return df

In [23]:
# 出现底背离金叉的时候，先不着急进场，等价格突破均线再进场，然后加各种过滤条件
def wait_chance(df,df_j,df_s):
    date_list = df_j[df_j['JCHA_FLAG']==1].Date.values.tolist()
    
    # 计算底背离后面走势，走出90bias>0的那天
    trade_flag = []
    for i in range(0,len(date_list)):
        if i ==len(date_list)-1:
            df1 = df[(df['Date']>=date_list[i])]
        else:
            df1 = df[(df['Date']>=date_list[i]) & (df['Date']<=date_list[i+1])]
        nice_date = df1[df1['5upcross60']>=0]['Date'].tolist()
        if len(nice_date):
            trade_flag.append(nice_date[0])
    df.loc[(df['Date'].isin(trade_flag)),'BUY_FLAG'] = 1
    
    # 再次筛选，涨幅不大，量能不大地突破均线的底背离
    # 去掉突破形式为一字板的情况,最高点等于最低点
    df.loc[((df['BUY_FLAG']==1) & (df['High']==df['Low'])),'BUY_FLAG'] = 0
    buy_date = df[df['BUY_FLAG']==1].Date.unique()
    ture_date =[]
    for day in buy_date:
        day_20 = (pd.to_datetime(day)-timedelta(days=30)).strftime('%Y-%m-%d')
#         day_10 = (pd.to_datetime(day)-timedelta(days=10)).strftime('%Y-%m-%d')
        
#         # 10天前看量能
#         df10 = df[((df['Date']<day) & (df['Date']>day_10))]
#         df10['vol_itude'] = df10['Volume']/df10['Vol_5']
        
        # 20天前看最大涨幅
        df20 = df[((df['Date']<=day) & (df['Date']>=day_20))]
        df20['min'] = df20['Low'].expanding().min()
        df20['max_chg'] = 100*(df20['Close'] / df20['min'] - 1)
        df20['vol_itude_20'] = df20['Volume']/df20['Vol_5']
        
        
        if (len(df20[df20['vol_itude_20']>2.3]) | len(df20[df20['max_chg']>22])):
            pass
        else:
            ture_date.append(day)
    df['trade_type'] = None
    if len(ture_date):
        df.loc[(df['Date'].isin(ture_date)),'trade_type'] = 1
    else:
        pass
#     buy_df = df[['ts_code','Date','trade_type','pct_chg','pct_chg_5','pct_chg_10','pct_chg_20','pct_chg_30','RSI','5upcross60','now_high_close','std_60','30_down','diff_MA60-MA90',
#                 ]]
    
    buy_df = df[['ts_code','Date','trade_type','pct_chg','pct_chg_5','pct_chg_10','pct_chg_20','pct_chg_30','RSI','5upcross60','now_high_close','std_60','30_down'
                 ,'alpha001','alpha005','alpha020','alpha025','alpha033','alpha039','alpha088','diff_MA60-MA90']]
    
    # 出现买入信号
    buy_df = buy_df[buy_df['trade_type']==1]
    
    # 各种筛选条件
    # RSI<67
    buy_df = buy_df[buy_df['RSI']<=67]
    
    # 当日涨幅要大于0.2%，小于8%
    buy_df = buy_df[buy_df['pct_chg']>0.2]
    buy_df = buy_df[buy_df['pct_chg']<8.0]
    
    # 当日回撤要大于-4%
    buy_df = buy_df[buy_df['now_high_close']>-4.0]
    
    # 30天内从最高点回撤要大于-6.20%,小于3.55%
    buy_df = buy_df[buy_df['30_down']>-6.20]
    buy_df = buy_df[buy_df['30_down']<3.55]
    
    # 60日内波动率要小于7%，大于1%
    buy_df = buy_df[buy_df['std_60']<7.0]
    buy_df = buy_df[buy_df['std_60']>1.0]
    
    # MA60与MA90的间隔要大于-12.2%
    buy_df = buy_df[buy_df['diff_MA60-MA90']>-12.2]
    
    # 一年半内的至今与当日比较最大回撤要>20%
#     buy_df = buy_df[buy_df['max_down']<-20]
    
#     buy_df.dropna(inplace=True)
    
    
    return buy_df

In [24]:
# 去极值+标准化
# 去极值，处理离群值，将超出变量特定百分位范围的数值替换为其特定百分位数值。
def winsor(x):
    if x.dropna().shape[0] != 0:
        x.loc[x < np.percentile(x.dropna(), 5)] = np.percentile(x.dropna(), 5)
        x.loc[x > np.percentile(x.dropna(), 95)] = np.percentile(x.dropna(), 95)
    else:
        x = x.fillna(0)
    return x
# 标准化
def standardize(x):
    return (x - x.mean()) / x.std()

In [25]:
# 接口
token = '572c7e13b431da6b7481ca0693db0b56fc83b14bd41055e52e74b459'
pro = ts.pro_api(token)

In [26]:
# 初始化数据
start = '20200520'
end = '20210622'
name_dict = {'trade_date':'Date','open':'Open','high':'High',
             'low':'Low','close':'Close','vol':'Volume',
             'amount':'Value',
            }

In [27]:
# 获取股票列表
df_code = get_allstock_code(exchange='', list_status='L', fields='ts_code, name')
# 只选要00，60开头的，不要st
df_code = df_code[df_code['name'].str.contains('ST') == False]
df_code = df_code[df_code['ts_code'].str.startswith('68') == False]
df_code = df_code[df_code['ts_code'].str.startswith('300') == False]
df_code.reset_index(drop=True,inplace=True)
df_code

,ts_code,name
0,000001.SZ,平安银行
1,000002.SZ,万科A
2,000004.SZ,国华网安
3,000006.SZ,深振业A
4,000008.SZ,神州高铁
...,...,...
2923,605398.SH,新炬网络
2924,605399.SH,晨光新材
2925,605488.SH,福莱新材
2926,605499.SH,东鹏饮料


In [28]:
# 获取符合形态条件的票
all_jchadata = pd.DataFrame()
for code in df_code.ts_code:
    print(code)
    df = get_stock_data(code, start, end)
    if(len(df)>200):
        df.rename(columns=name_dict,inplace=True) # 列重命名
        df = fuquan(df)
        df_j,df_s = MACD_Deviation(df)
        buy_df = wait_chance(df,df_j,df_s)
        all_jchadata = all_jchadata.append(buy_df)
all_jchadata

000001.SZ
000002.SZ
000004.SZ
000006.SZ
000008.SZ
000009.SZ
000010.SZ
000011.SZ
000012.SZ
000014.SZ
000016.SZ
000019.SZ
000020.SZ
000021.SZ
000023.SZ
000025.SZ
000026.SZ
000027.SZ
000028.SZ
000029.SZ
000030.SZ
000031.SZ
000032.SZ
000034.SZ
000035.SZ
000036.SZ
000037.SZ
000038.SZ
000039.SZ
000040.SZ
000042.SZ
000045.SZ
000046.SZ
000048.SZ
000049.SZ
000050.SZ
000055.SZ
000056.SZ
000058.SZ
000059.SZ
000060.SZ
000061.SZ
000062.SZ
000063.SZ
000065.SZ
000066.SZ
000068.SZ
000069.SZ
000070.SZ
000078.SZ
000088.SZ
000089.SZ
000090.SZ
000096.SZ
000099.SZ
000100.SZ
000150.SZ
000151.SZ
000153.SZ
000155.SZ
000156.SZ
000157.SZ
000158.SZ
000159.SZ
000166.SZ
000301.SZ
000333.SZ
000338.SZ
000400.SZ
000401.SZ
000402.SZ
000403.SZ
000404.SZ
000407.SZ
000408.SZ
000409.SZ
000411.SZ
000413.SZ
000415.SZ
000416.SZ
000417.SZ
000419.SZ
000420.SZ
000421.SZ
000422.SZ
000423.SZ
000425.SZ
000426.SZ
000428.SZ
000429.SZ
000430.SZ
000488.SZ
000498.SZ
000501.SZ
000503.SZ
000504.SZ
000505.SZ
000506.SZ
000507.SZ
000510.SZ


002426.SZ
002428.SZ
002429.SZ
002430.SZ
002431.SZ
002432.SZ
002433.SZ
002434.SZ
002435.SZ
002436.SZ
002437.SZ
002438.SZ
002439.SZ
002440.SZ
002441.SZ
002442.SZ
002443.SZ
002444.SZ
002445.SZ
002446.SZ
002448.SZ
002449.SZ
002451.SZ
002452.SZ
002453.SZ
002454.SZ
002455.SZ
002456.SZ
002457.SZ
002458.SZ
002459.SZ
002460.SZ
002461.SZ
002462.SZ
002463.SZ
002465.SZ
002466.SZ
002467.SZ
002468.SZ
002469.SZ
002471.SZ
002472.SZ
002474.SZ
002475.SZ
002476.SZ
002478.SZ
002479.SZ
002480.SZ
002481.SZ
002482.SZ
002483.SZ
002484.SZ
002485.SZ
002486.SZ
002487.SZ
002488.SZ
002489.SZ
002490.SZ
002491.SZ
002492.SZ
002493.SZ
002494.SZ
002495.SZ
002497.SZ
002498.SZ
002500.SZ
002502.SZ
002503.SZ
002505.SZ
002506.SZ
002507.SZ
002508.SZ
002510.SZ
002511.SZ
002512.SZ
002514.SZ
002515.SZ
002516.SZ
002517.SZ
002518.SZ
002519.SZ
002520.SZ
002521.SZ
002522.SZ
002523.SZ
002524.SZ
002526.SZ
002527.SZ
002528.SZ
002530.SZ
002531.SZ
002532.SZ
002533.SZ
002534.SZ
002536.SZ
002537.SZ
002538.SZ
002539.SZ
002540.SZ
002541.SZ


ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=10809): Read timed out. (read timeout=30)

In [15]:
all_data = all_jchadata[all_jchadata['Date']>='2021-06-11']
all_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,diff_MA60-MA90
254,002387.SZ,2021-06-11,1,2.67,NaN,NaN,NaN,NaN,62.995252,1.13,-0.31,5.6818,-1.33,0.523207,-0.027191,-0.235781,0.31250,0.181467,-1.542857,0.239382,-2.48
261,600339.SH,2021-06-16,1,3.09,NaN,NaN,NaN,NaN,62.745795,0.37,-2.56,3.2870,0.66,0.715768,-0.523143,-0.256777,0.02449,0.117424,-1.346304,0.193182,2.24


In [15]:
all_data['Date'] = all_data['Date'].apply(date_to_str)

In [16]:
ticker_list = all_data['ts_code'].unique()

In [17]:
add_date = pd.DataFrame()
for code in ticker_list:
    date_list = all_data[all_data['ts_code']==code]['Date'].tolist()
    for date in date_list:
        df = pro.daily_basic(ts_code=code,trade_date=date, fields='')
        add_date = add_date.append(df)
add_date

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,002387.SZ,20210611,9.62,0.9904,1.6444,0.95,64.6306,3662.8730,0.8929,3.8310,3.6170,None,None,136766.3046,136721.3346,82343.1611,1.315692e+06,1.315259e+06
0,600339.SH,20210616,3.00,2.2095,7.9477,2.42,19.5895,11.9325,0.6769,0.2369,0.2251,1.4667,1.4667,558314.7471,558314.7471,155218.0662,1.674944e+06,1.674944e+06


In [18]:
all_features_data = pd.merge(all_data,add_date,how='left',left_on = ['ts_code','Date'],right_on = ['ts_code','trade_date'])
all_features_data

,ts_code,Date,trade_type,pct_chg,pct_chg_5,pct_chg_10,pct_chg_20,pct_chg_30,RSI,5upcross60,now_high_close,std_60,30_down,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,diff_MA60-MA90,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,002387.SZ,20210611,1,2.67,NaN,NaN,NaN,NaN,62.978134,1.06,-0.31,5.6798,-1.33,0.518219,-0.025586,-0.222626,0.318548,0.183521,-1.823504,0.250936,-2.47,20210611,9.62,0.9904,1.6444,0.95,64.6306,3662.8730,0.8929,3.8310,3.6170,None,None,136766.3046,136721.3346,82343.1611,1.315692e+06,1.315259e+06
1,600339.SH,20210616,1,3.09,NaN,NaN,NaN,NaN,62.715589,0.35,-2.56,3.2678,0.66,0.718000,-0.543300,-0.254574,0.023715,0.113971,-1.343396,0.205882,2.24,20210616,3.00,2.2095,7.9477,2.42,19.5895,11.9325,0.6769,0.2369,0.2251,1.4667,1.4667,558314.7471,558314.7471,155218.0662,1.674944e+06,1.674944e+06


In [19]:
all_features_data['Date'] = all_features_data['Date'].apply(str_to_date)

In [20]:
my_list = ['RSI',
 '5upcross60',
 'now_high_close',
 'std_60',
 '30_down',
 'diff_MA60-MA90',
 'close',
 'turnover_rate',
 'volume_ratio',
 'pe',
 'pe_ttm',
 'pb',
 'ps',
 'dv_ratio',
 'total_mv',
 'alpha001',
 'alpha005',
 'alpha020',
 'alpha025',
 'alpha033',
 'alpha039',
 'alpha088',
 'pct_chg']

In [21]:
all_features_data.set_index(['Date','ts_code'],inplace=True)
X1 = all_features_data[my_list]
X1

,,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,volume_ratio,pe,pe_ttm,pb,ps,dv_ratio,total_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg
Date,ts_code,,,,,,,,,,,,,,,,,,,,,,,
2021-06-11,002387.SZ,62.978134,1.06,-0.31,5.6798,-1.33,-2.47,9.62,0.9904,0.95,64.6306,3662.8730,0.8929,3.8310,None,1.315692e+06,0.518219,-0.025586,-0.222626,0.318548,0.183521,-1.823504,0.250936,2.67
2021-06-16,600339.SH,62.715589,0.35,-2.56,3.2678,0.66,2.24,3.00,2.2095,2.42,19.5895,11.9325,0.6769,0.2369,1.4667,1.674944e+06,0.718000,-0.543300,-0.254574,0.023715,0.113971,-1.343396,0.205882,3.09


In [22]:
X1.fillna(0,inplace=True)

In [23]:
X0 = pd.read_csv('basic.csv',encoding='gbk',index_col=[0,1])
X2 = X1.append(X0)
X2 =  X2.apply(lambda x:  winsor(x), axis=0)
X2 =  X2.apply(lambda x: standardize(x), axis=0)
X3 = X2.loc[X1.index]
X3

,,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,volume_ratio,pe,pe_ttm,pb,ps,dv_ratio,total_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg
Date,ts_code,,,,,,,,,,,,,,,,,,,,,,,
2021-06-11,002387.SZ,0.649468,1.734460,0.753923,1.737582,-0.332527,0.387903,-0.282755,-0.525873,-0.833764,0.189731,2.993436,-1.084616,0.037219,-0.893230,0.012116,-1.172858,1.091727,-0.135386,0.294072,-0.595638,-1.572916,-0.631375,0.955738
2021-06-16,600339.SH,0.572554,-0.291468,-2.345277,-0.173320,0.917851,1.699899,-1.037738,0.317478,1.909917,-0.522681,-0.682216,-1.103973,-0.923073,0.358267,0.251365,-0.177974,-1.387893,-0.317866,-1.728549,-1.095163,-1.110208,-0.816844,1.331796


In [24]:
# 下载本地模型
model = joblib.load(filename="svm.pkl")
# Ypredict = model1.predict(X3)
Ypredict = np.where(model.decision_function(X3)>=0.06703898246205468,1,0)
Ypredict

array([0, 0])

In [25]:
X3['signal'] = Ypredict
X3

,,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,volume_ratio,pe,pe_ttm,pb,ps,dv_ratio,total_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg,signal
Date,ts_code,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-11,002387.SZ,0.649468,1.734460,0.753923,1.737582,-0.332527,0.387903,-0.282755,-0.525873,-0.833764,0.189731,2.993436,-1.084616,0.037219,-0.893230,0.012116,-1.172858,1.091727,-0.135386,0.294072,-0.595638,-1.572916,-0.631375,0.955738,0
2021-06-16,600339.SH,0.572554,-0.291468,-2.345277,-0.173320,0.917851,1.699899,-1.037738,0.317478,1.909917,-0.522681,-0.682216,-1.103973,-0.923073,0.358267,0.251365,-0.177974,-1.387893,-0.317866,-1.728549,-1.095163,-1.110208,-0.816844,1.331796,0


In [26]:
output_df = X3[X3['signal']==1]
output_df

,,RSI,5upcross60,now_high_close,std_60,30_down,diff_MA60-MA90,close,turnover_rate,volume_ratio,pe,pe_ttm,pb,ps,dv_ratio,total_mv,alpha001,alpha005,alpha020,alpha025,alpha033,alpha039,alpha088,pct_chg,signal
Date,ts_code,,,,,,,,,,,,,,,,,,,,,,,,


In [48]:
output_stock_list = output_df.reset_index()['ts_code'].to_list()
output_stock_list

['600498.SH', '600523.SH', '603677.SH', '603920.SH']

In [31]:
df_code[df_code.ts_code == '603661.SH'].index

Int64Index([2650], dtype='int64')